In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import mixed_precision

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
CLASSES = ["acne", "pigmentation", "wrinkles"]
DATA_ROOT = "/content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/"

df = pd.read_csv(os.path.join(DATA_ROOT, "labels.csv"))
df["filename"] = df["filename"].apply(lambda x: os.path.join(DATA_ROOT, x))

In [ ]:
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42, stratify=df[CLASSES])
train_df, val_df = train_test_split(train_val_df, test_size=0.15, random_state=42, stratify=train_val_df[CLASSES])

pos_counts = train_df[CLASSES].sum().values

In [ ]:
#Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.3),
    layers.RandomContrast(0.2),
], name="data_augmentation")

#Create tf.data Pipelines
def parse_function(filename, labels):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image_decoded, tf.float32)
    image_resized = tf.image.resize(image, IMG_SIZE)
    return image_resized, labels

def create_dataset(df, batch_size, augment=False, cache_file=None):
    dataset = tf.data.Dataset.from_tensor_slices(
        (df["filename"].values, df[CLASSES].values.astype(np.float32))
    )
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
                              num_parallel_calls=tf.data.AUTOTUNE)

    if cache_file:
        dataset = dataset.cache(cache_file)
    else:
        dataset = dataset.cache()

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

#Create the datasets with disk caching
train_cache_file = os.path.join(DATA_ROOT, 'train_cache_densenet')
val_cache_file = os.path.join(DATA_ROOT, 'val_cache_densenet')

train_ds = create_dataset(train_df, BATCH_SIZE, augment=True, cache_file=train_cache_file)
val_ds = create_dataset(val_df, BATCH_SIZE, augment=False, cache_file=val_cache_file)
test_ds = create_dataset(test_df, BATCH_SIZE, augment=False)

print("tf.data pipelines created successfully with disk caching enabled.")

tf.data pipelines created successfully with disk caching enabled.


In [ ]:
def build_densenet(input_shape, num_classes):
    base_model = DenseNet121(
        include_top=False,
        input_shape=input_shape,
        weights="imagenet"
    )

    inputs = layers.Input(shape=input_shape)

    x = tf.keras.applications.densenet.preprocess_input(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, activation="sigmoid", dtype='float32')(x)
    model = Model(inputs, outputs, name="densenet121_model")
    return model

tf.keras.backend.clear_session()
densenet_model = build_densenet(input_shape=IMG_SIZE + (3,), num_classes=len(CLASSES))
densenet_model.summary()

Model: "densenet121_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ add (Add)                       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_1 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet121 (Functional)        │ (None, 7, 7, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │         3,075 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,040,579 (26.86 MB)

 Trainable params: 6,956,931 (26.54 MB)

 Non-trainable params: 83,648 (326.75 KB)

In [ ]:
#Custom Weighted Binary Cross-Entropy Loss
def weighted_bce(y_true, y_pred, smooth=0.05):
    y_true = y_true * (1.0 - smooth) + 0.5 * smooth
    bce = tf.keras.backend.binary_crossentropy(y_true, y_pred)
    pos = tf.constant(pos_counts, dtype=tf.float32)
    neg = len(train_df) - pos
    w_pos = neg / tf.maximum(pos, 1.0)
    w_neg = tf.ones_like(pos)
    weights = y_true * w_pos + (1.0 - y_true) * w_neg
    return tf.reduce_mean(bce * weights)

#Callbacks
DENSENET_MODEL_PATH = os.path.join(DATA_ROOT, "densenet_skin_model.keras")
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=7, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_auc", mode="max", factor=0.2, patience=3, min_lr=1e-6),
    tf.keras.callbacks.ModelCheckpoint(DENSENET_MODEL_PATH, monitor="val_auc", mode="max", save_best_only=True)
]

#STAGE 1: FEATURE EXTRACTION
print("\nStage 1: Training the classification head...")
densenet_model.get_layer("densenet121").trainable = False
densenet_model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-4),
    loss=weighted_bce,
    metrics=["acc", "auc"]
)
history_head = densenet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    verbose=1
)

#STAGE 2: FINE-TUNING
print("\nStage 2: Unfreezing and fine-tuning the entire model...")
densenet_model.get_layer("densenet121").trainable = True
densenet_model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-5, weight_decay=1e-4),
    loss=weighted_bce,
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="acc", threshold=0.5),
        tf.keras.metrics.AUC(name="auc", multi_label=True),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)
history_fine_tune = densenet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=callbacks,
    initial_epoch=len(history_head.history['loss']),
    verbose=1
)


Stage 1: Training the classification head...
Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 313s 2s/step - acc: 0.3000 - auc: 0.5428 - loss: 1.2201 - val_acc: 0.5015 - val_auc: 0.8472 - val_loss: 1.0761
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 11s 99ms/step - acc: 0.3918 - auc: 0.6832 - loss: 1.0999 - val_acc: 0.5077 - val_auc: 0.8441 - val_loss: 1.0263
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 20s 172ms/step - acc: 0.4283 - auc: 0.7465 - loss: 1.0474 - val_acc: 0.5155 - val_auc: 0.8529 - val_loss: 0.9931
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 12s 101ms/step - acc: 0.4477 - auc: 0.7846 - loss: 1.0072 - val_acc: 0.5046 - val_auc: 0.8511 - val_loss: 0.9698
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 11s 97ms/step - acc: 0.4622 - auc: 0.7982 - loss: 0.9812 - val_acc: 0.5046 - val_auc: 0.8519 - val_loss: 0.9547
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 12s 102ms/step - acc: 0.4576 - auc: 0.7997 - loss: 0.9725 - val_acc: 0.5062 - val_auc: 0.8544 - val_loss: 0.9428
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
#Load the Best Saved Model
print(f"Loading best model from: {DENSENET_MODEL_PATH}")
loaded_model = tf.keras.models.load_model(
    DENSENET_MODEL_PATH,
    custom_objects={"weighted_bce": weighted_bce}
)
print("Model loaded successfully!")

#Evaluate on the Unseen Test Set
print("\nEvaluating the final model on the test set...")
test_results = loaded_model.evaluate(test_ds, return_dict=False)

print("\nFinal Test Set Evaluation Results")
for metric, value in zip(loaded_model.metrics_names, test_results):
    print(f"{metric}: {value:.4f}")

#Get Model Size
file_size_bytes = os.path.getsize(DENSENET_MODEL_PATH)
file_size_mb = file_size_bytes / (1024 * 1024)
print(f"\nModel Size on Disk: {file_size_mb:.2f} MB")

#Get Training Time
if 'history_head' in locals() and 'history_fine_tune' in locals():
    total_epochs_ran = len(history_head.history['loss']) + len(history_fine_tune.history['loss'])
    print(f"Total Epochs Trained: {total_epochs_ran}")
    print("(Check Colab cell execution time for total training duration)")

Loading best model from: /content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/densenet_skin_model.keras
Model loaded successfully!

Evaluating the final model on the test set...
24/24 ━━━━━━━━━━━━━━━━━━━━ 34s 743ms/step - acc: 0.9426 - auc: 0.9754 - loss: 0.7059 - precision: 0.8079 - recall: 0.9132

Final Test Set Evaluation Results
loss: 0.6671
compile_metrics: 0.9487

Model Size on Disk: 81.64 MB
Total Epochs Trained: 44
(Check Colab cell execution time for total training duration)
